In [2]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
# Install required packages (if not installed)
!pip install datasets

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
from datasets import load_dataset
from sklearn.model_selection import train_test_split

# Define file path (Update as needed)
json_file_path = "/content/drive/MyDrive/BHBFC_chatbot/dataset/qa_pairs_mistral.json"  # Modify this path

# Load the dataset from JSON file
dataset = load_dataset('json', data_files=json_file_path, split='train')

# Split dataset into train (90%) and eval (10%)
train_test_split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# Print dataset details
print(f"✅ Train Dataset Size: {len(train_dataset)}")
print(f"✅ Eval Dataset Size: {len(eval_dataset)}")
print("🔹 Sample Data from Training Set:")
print(train_dataset[0])  # Print first example


Mounted at /content/drive


Generating train split: 0 examples [00:00, ? examples/s]

✅ Train Dataset Size: 41
✅ Eval Dataset Size: 5
🔹 Sample Data from Training Set:
{'question': 'What is considered when approving a primary application?', 'answer': "Answer : List of documents to be submitted along with the primary application:\no Attested photocopy of design and design permit of under construction/under construction\napproved by the appropriate authority.\no Ownership documents, namazari khatiyan, DCR, attested photocopy of Halson's rent\nreceipt, etc.\no To facilitate the inspection of the construction site by this organization , 2 copies of\nhand-drawn route map (with the signature of the applicant) on the tracing paper\nmentioning important structures in the vicinity along with the details of the road leading\nto the proposed construction site.\nThe temporary application form can be collected free of cost from the concerned\nbranch office or downloaded from the corporation's website and filled it properly\nand submitted."}


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()


In [14]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `sadas` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate wh

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "gpt2"  # You can also use "distilgpt2" for a smaller model

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Fine-tune with your chatbot data


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [18]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from transformers import DataCollatorForSeq2Seq
import torch

In [20]:
train_dataset[:2]

{'question': ['What is considered when approving a primary application?',
  'Can someone living abroad such as Malaysia or Canada build a house with a loan from\nBangladesh House Building Finance Corporation?'],
 'answer': ["Answer : List of documents to be submitted along with the primary application:\no Attested photocopy of design and design permit of under construction/under construction\napproved by the appropriate authority.\no Ownership documents, namazari khatiyan, DCR, attested photocopy of Halson's rent\nreceipt, etc.\no To facilitate the inspection of the construction site by this organization , 2 copies of\nhand-drawn route map (with the signature of the applicant) on the tracing paper\nmentioning important structures in the vicinity along with the details of the road leading\nto the proposed construction site.\nThe temporary application form can be collected free of cost from the concerned\nbranch office or downloaded from the corporation's website and filled it properly\n

In [30]:
# Path to your dataset on Google Drive
file_path = '/content/drive/MyDrive/BHBFC_chatbot/dataset/qa_pairs_mistral.json'  # Modify this to your correct file path





# Step 2: Load the dataset
dataset = load_dataset('json', data_files=file_path)

# Extract questions and answers
questions = dataset['train']['question']
answers = dataset['train']['answer']

# Combine the 'question' and 'answer' into a single 'text' field in the format "Question: <question> Answer: <answer>"
def format_data(example):
    return {"text": f"Question: {example['question']} Answer: {example['answer']}"}

# Apply this formatting to the entire train dataset
train_dataset = dataset['train'].map(format_data, remove_columns=['question', 'answer'])

# Check the first entry after formatting
print(train_dataset[0])  # This should print the formatted question-answer pair

# Optionally, split the dataset into training and evaluation datasets (90% train, 10% eval)
train_dataset, eval_dataset = train_dataset.train_test_split(test_size=0.1).values()

# Check the size of the split datasets
print(f"Train Dataset Size: {len(train_dataset)}")
print(f"Eval Dataset Size: {len(eval_dataset)}")

Map:   0%|          | 0/46 [00:00<?, ? examples/s]

{'text': 'Question: I have a piece of land. What loan facility can I get from Bangladesh House Building\nFinance Corporation? Answer: Page 1 of 10\nQuestions and Answers to House Building Loans\nQuestion : I have a piece of land. What loan facility can I get from Bangladesh House Building'}
Train Dataset Size: 41
Eval Dataset Size: 5


In [32]:
# Step 3: Load GPT-2 model and tokenizer
model_name = "gpt2"  # You can choose other models as well
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set the pad token to be the same as the EOS token
tokenizer.pad_token = tokenizer.eos_token

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Tokenize the datasets
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/41 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [35]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# Data collator for causal language modeling (GPT-2)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # GPT-2 is a causal language model, so we don't use MLM (masked language modeling)
)

# Step 5: Define Training Arguments
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save the model and logs
    overwrite_output_dir=True,
    num_train_epochs=15,  # You can adjust the number of epochs
    per_device_train_batch_size=4,  # Batch size per GPU/TPU core
    per_device_eval_batch_size=8,  # Evaluation batch size
    logging_dir='./logs',  # Directory to store logs
    logging_steps=500,  # Log every 500 steps
    save_steps=1000,  # Save the model every 1000 steps
    evaluation_strategy="steps",  # Evaluate during training
    eval_steps=500,  # Evaluate every 500 steps
    load_best_model_at_end=True,  # Load the best model during evaluation
    weight_decay=0.01,  # Regularization to avoid overfitting
    push_to_hub=False,  # Set to True if you want to push the model to Hugging Face hub
)

# Step 6: Set Up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-35-38c46cabee03>:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [36]:
# Start training
trainer.train()
# Save the fine-tuned model
trainer.save_model('./fine_tuned_gpt2')
# Evaluate the model on the evaluation dataset
eval_results = trainer.evaluate()
print(eval_results)


Step,Training Loss,Validation Loss


{'eval_loss': 3.3149311542510986, 'eval_runtime': 0.0974, 'eval_samples_per_second': 51.321, 'eval_steps_per_second': 10.264, 'epoch': 15.0}


In [37]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained('./fine_tuned_gpt2')
tokenizer = AutoTokenizer.from_pretrained('gpt2')


In [38]:
def generate_answer(question, model, tokenizer, max_length=50):
    # Tokenize the input question
    input_ids = tokenizer.encode(question, return_tensors='pt')

    # Generate a response using the fine-tuned model
    output_ids = model.generate(input_ids, max_length=max_length, num_return_sequences=1, no_repeat_ngram_size=2, top_p=0.95, temperature=0.7)

    # Decode the generated response
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return answer


In [43]:
question = "What is the interest rate on loan money borrowed from BHBFC"
answer = generate_answer(question, model, tokenizer)
print("Generated Answer:", answer)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Answer: What is the interest rate on loan money borrowed from BHBFC? Answer: Answer : Loans are given only for the construction of residential buildings. However, in case of commercial buildings, the loan can be obtained from
the corporation.
Question :
